In [ ]:
# Insert amout of pairs
N = 10
AB_df = flowlogs_df.loc[flowlogs_df["src_ip"] > flowlogs_df["dst_ip"]]
BA_df = flowlogs_df.loc[flowlogs_df["dst_ip"] > flowlogs_df["src_ip"]].rename(columns={"src_ip":"dst_ip", "dst_ip":"src_ip", "ABPackets":"BAPackets", "BAPackets":"ABPackets"})
ordered_df = AB_df.append(BA_df)
    
res_df = ordered_df.groupby(['src_ip', 'dst_ip']).agg({"Total_Packets":'sum', "ABPackets":'sum', "BAPackets":'sum'}).nlargest(N, 'Total_Packets').sort_values(by=["Total_Packets"], ascending=False).reset_index()

ip1, ip2, stream1, stream2 = res_df.iloc[0][["src_ip", "dst_ip", "ABPackets", "BAPackets"]]
A, B, AB, BA = (ip1, ip2, stream1, stream2) if stream1 > stream2 else (ip2, ip1, stream2, stream1)

print("The top talking pair is (%s, %s)," % (A,  B))
print("While %.2f%% of the traffic goes from %s to %s\n\n\n" % (100*AB/(AB + BA), A,  B))

# Plot Graph

plt.figure(figsize=(20,10))

edges = [(res_df['src_ip'].iloc[i], res_df['dst_ip'].iloc[i], {'weight': res_df['Total_Packets'].iloc[i]}) for i in range(len(res_df.index))]

G = nx.OrderedGraph()
G.add_edges_from(edges)

# positions for all nodes
pos = nx.circular_layout(G) 

# labels
nx.draw_networkx_labels(G, pos)

# edge widths
widths = scale(res_df['Total_Packets'].values, 10)

# edges
nx.draw_networkx_edges(G, pos, width=widths, alpha=0.3)

# node sizes
sizes = scale([elem[1] for elem in G.degree(weight='weight')], 2000)

nx.draw_networkx_nodes(G, pos, node_color = 'orange', node_size = sizes)

# weights
labels = nx.get_edge_attributes(G, 'weight')
for key, value in labels.items():
    labels[key] = number_format(value)
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

plt.axis('off')
plt.title('Top-%s Talking Pairs' % N, fontsize=30)
plt.show()